In [1]:
import cv2
import numpy as np
from robocup import localization as loc
from robocup import field
import math

In [3]:
f = cv2.imread('slike/8.jpg')
a, b = loc.polar_line2cart(400, 0)
f = loc.draw_rect_line(f, a, b, -1, 190, 50)
cv2.imshow("sjkj", f)
cv2.waitKey(0)
cv2.destroyAllWindows()

-10000 4000000.0 400.0
coeff -0.9999999949999999
[[210   0]
 [160   0]
 [159 499]
 [209 499]]
[[210   0]
 [160   0]
 [159 499]
 [209 499]]


In [57]:
%timeit x, y = find_xy(f, w, 50, 0.2, 0.3)

100 loops, best of 3: 2.15 ms per loop


In [4]:
w = 500
scale = 500
f = cv2.imread('slike/3.jpg', 0)        


x, y = loc.find_xy(f, w, 50, 0.2, 0.3)
print(x, y)
    

            

f = loc.draw_cross(f, 250, 250, 20, 255)   
cv2.imshow("sjkj", f)
cv2.waitKey(0)
cv2.destroyAllWindows()

1381.0485871204426 1361.0845871204426


In [51]:
def find_xy(f, w, line_sens, lines_min_dist, right_angle_deviation):
    x_coor = -1
    y_coor = -1
    lines = cv2.HoughLines(f,2,np.pi/180*2,line_sens)
        
    if lines is not None:
        nms_lines = []
        found = False
        for l in lines:
            for rho, theta in l:
                l = l[0]
                if loc.lines_d (l, nms_lines, w*1.41, math.pi) > lines_min_dist:
                    if not found or loc.is_on_right(nms_lines[0], l, right_angle_deviation):
                        nms_lines.append(l)
                        found = True
                        loc.draw_line_polar(f, l)


                        dist = loc.pl_dist_polar2(w/2, w/2, rho, theta)
                        dist2 = dist / field.scale_field
                        if is_y(l) and x_coor == -1:
                            if loc.side_x(int(w/2), int(w/2), rho, theta, f):
                                x_coor = field.x - dist2
                            else:
                                x_coor = dist2
                        elif y_coor == -1:   
                            if not loc.side_y(int(w/2), int(w/2), rho, theta, f):
                                y_coor = field.y - dist2
                            else:
                                y_coor = dist2
                        
                        if x_coor!= -1 and y != -1:
                            return x_coor, y_coor
                        
                        #print("x = ", x_coor, "y = ", y_coor, " dist= ", dist, " dist2 = ", dist2)#, end = '\r') #, " rho=", rho, " theta=", theta, " a=", a, " b=", b, " pres=", x0, " rho=", rho, " theta = ", theta, " end " , end='\r')
    return x_coor, y_coor

In [111]:
def lines_d (l_test, nms_lines, u_range, v_range):
    mini = 1000;
    #print(nms_lines)
    for l in nms_lines:
        #l = l[0
        temp = dist_rho(l, l_test) / u_range + dist_angl(l_test[1], l[1]) / v_range
        #print("ls" ,l_test, l, dist_rho(l, l_test) / u_range,   dist_angl(l_test[1], l[1]) / v_range)
        if temp < mini:
            mini = temp
    
    #(mini)
    return mini

In [87]:
def dist_angl(a, b):
    x = abs(a-b)
    y = abs(math.pi - a - b)
    if x < y:
        return x
    return y

In [88]:
def dist_rho(l1, l2):
    if l1[0]*l2[0] < 0:
        if l1[1] > math.pi/2 and l2[1] < math.pi/2 or l2[1] > math.pi/2 and l1[1] < math.pi/2:
            return abs(abs(l1[0]) - abs(l2[0]))
    return abs(l1[0] - l2[0])        

In [117]:
def is_on_right(l0, l, d):
    #print("jfsgjsjj")
    t0 = l0[1]
    t1 = l[1]
    if t0 > math.pi:
        t0 -= math.pi
    if t1 > math.pi:
        t1 -= math.pi
        
    dist = dist_angl(t0, t1)
    print("deist = ", dist, l0, l)
    if dist < d or dist > math.pi/2 -d and dist < math.pi/2 + d:
        return True
    return False

In [86]:
5 % 3

2

In [122]:
def draw_line_polar(img, l):
    rho = l[0]
    theta = l[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),255,2)
    
    return img

In [10]:
def is_y(l):
    theta = l[1]
    if theta < math.pi/4 or theta > math.pi/4*3:
        return True
    return False